In [1]:
from datasets import load_dataset
import dspy
import openai
import os
import re
import pandas as pd
import ast
import json
from dotenv import load_dotenv
import numpy as np
import time

In [2]:
load_dotenv()

True

In [3]:

openai.api_key = os.getenv('OPENAI_API_KEY')
openai.organization = os.getenv('OPENAI_ORGANIZATION')


In [4]:
lm = dspy.LM('openrouter/anthropic/claude-3.5-sonnet', temperature=0, max_tokens=1024)
dspy.configure(lm=lm)

In [5]:
ds = pd.read_json('../preprocessing/train_dev_test_data/ner/fewnerd_sample_test.json', encoding_errors='replace')
ds = ds.to_dict('records')


In [6]:
ds[0]

{'id': 1,
 'text': 'In the early 1930s the band moved to the Grill Room of the Taft Hotel in New York ; the band was renamed ``George Hall and His Hotel Taft Orchestra``.',
 'label': [{'Grill Room': 'BUILDING'},
  {'Taft Hotel': 'BUILDING'},
  {'New York': 'LOCATION'},
  {'George Hall and His Hotel Taft Orchestra': 'ORGANIZATION'}],
 'dataset': 'few_nerd',
 'entity': ['Grill Room',
  'Taft Hotel',
  'New York',
  'George Hall and His Hotel Taft Orchestra']}

In [7]:
def remove_space(text):
    """Clean up spacing and formatting in dialogue text."""
    lines = text.split('\n')
    cleaned_lines = []
    
    for line in lines:
        # Remove multiple spaces
        cleaned = ' '.join(line.split())
        
        # Fix spacing around punctuation
        cleaned = re.sub(r'\s+([.,!?:;])', r'\1', cleaned)
        cleaned = re.sub(r'([.,!?:;])\s+', r'\1 ', cleaned)
        
        # Fix contractions
        cleaned = re.sub(r'\s*\'\s*s\b', "'s", cleaned)
        cleaned = re.sub(r'\s*n\s*\'\s*t\b', "n't", cleaned)
        cleaned = re.sub(r'\s*\'\s*ve\b', "'ve", cleaned)
        cleaned = re.sub(r'\s*\'\s*re\b', "'re", cleaned)
        cleaned = re.sub(r'\s*\'\s*ll\b', "'ll", cleaned)
        cleaned = re.sub(r'\s*\'\s*d\b', "'d", cleaned)
        cleaned = re.sub(r'\s*\'\s*m\b', "'m", cleaned)
        
        # Fix spaces around parentheses
        cleaned = re.sub(r'\(\s+', '(', cleaned)
        cleaned = re.sub(r'\s+\)', ')', cleaned)
        
        # Remove leading/trailing whitespace
        cleaned = cleaned.strip()
        
        cleaned_lines.append(cleaned)
        
    return '\n'.join(cleaned_lines)


In [8]:
examples = [
    dspy.Example({ 
                  "text" : str(r["text"]), 
                  "label": str(r['label'])
                }
                  ).with_inputs("text")
    
    for r in ds
    ]


In [203]:
example = examples[0]
for k, v in example.items():
    print(f"\n{k.upper()}:\n")
    print(v)



TEXT:

In the early 1930s the band moved to the Grill Room of the Taft Hotel in New York ; the band was renamed ``George Hall and His Hotel Taft Orchestra``.

LABEL:

[{'Grill Room': 'BUILDING'}, {'Taft Hotel': 'BUILDING'}, {'New York': 'LOCATION'}, {'George Hall and His Hotel Taft Orchestra': 'ORGANIZATION'}]


In [9]:
def calculate_f1_ent(gold_entities, predicted_entities):
    """
    Calculates the F1 score given the true labels and predicted labels.
    """
    # print("Input types:")
    # print(f"gold_entities type: {type(gold_entities)}")
    # print(f"predicted_entities type: {type(predicted_entities)}")
    
    if predicted_entities is None:
        return 0.0, 0.0, 0.0

    true_entities = {}
    pred_entities = {}
    
    # Convert to empty list if NaN
    def handle_nan(entities):
        # print(f"Handling NaN for: {type(entities)}")
        # If it's already a list, return it as is
        if isinstance(entities, list):
            return entities
        # Handle pandas/numpy types
        if isinstance(entities, (pd.Series, np.ndarray)):
            nan_check = pd.isna(entities)
            if isinstance(nan_check, (pd.Series, np.ndarray)):
                if nan_check.any():
                    return "[]"
            elif nan_check:
                return "[]"
        # Handle single values
        elif pd.isna(entities):
            return "[]"
        return entities

    # Handle NaN cases
    gold_entities = handle_nan(gold_entities)
    predicted_entities = handle_nan(predicted_entities)
            
    # Parse strings if needed
    if isinstance(gold_entities, str):
        gold_entities = ast.literal_eval(gold_entities)
    if isinstance(predicted_entities, str):
        predicted_entities = ast.literal_eval(predicted_entities)

    # Process gold entities
    # print(gold_entities)
    for entity in gold_entities:
        # print(entity)
        if isinstance(entity, str):
            entity = ast.literal_eval(entity)
        if entity.get('text') is not None:
            true_entities[entity["text"]] = entity['value']
        else:
            for key, value in entity.items():
                true_entities[key] = value
    
    # Process predicted entities
    for entity in predicted_entities:
        if isinstance(entity, str):
            entity = ast.literal_eval(entity)
        if entity.get('text') is not None:  
            pred_entities[entity['text']] = entity['value']
        else:
            for key, value in entity.items():
                pred_entities[key] = value

    # Calculate metrics
    true_positives = sum(1 for text in true_entities if text in pred_entities and true_entities[text] == pred_entities[text])
    false_positives = sum(1 for text in pred_entities if text not in true_entities)
    false_negatives = sum(1 for text in true_entities if text not in pred_entities)

    if true_positives == 0:
        return 0.0, 0.0, 0.0

    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1_score = 2 * (precision * recall) / (precision + recall)

    return precision, recall, f1_score

In [10]:
def extract_prediction(pred):
    matches = re.findall(r"\[\{.*\}\]", pred)
    # print(matches)
    parsed_answer = matches[-1] if matches else ""
    if parsed_answer == "":
        return {}
    parsed_answer = ast.literal_eval(parsed_answer)
    return parsed_answer


In [11]:
def eval_metric(true, prediction, trace=None, provide_traceback=True):
    pred = prediction.label
    
    matches = re.findall(r"\[\{.*\}\]", pred)
    # print(matches)
    parsed_answer = matches[-1] if matches else ""
    if parsed_answer == "":
        return 0.0
    parsed_answer = ast.literal_eval(parsed_answer)
    # print(type(parsed_answer))
    gold_entities = ast.literal_eval(true.label)
    # print(parsed_answer)
    precision, recall, f1_score = calculate_f1_ent(gold_entities=gold_entities, predicted_entities= parsed_answer)
    return f1_score

In [12]:
from dspy.evaluate import Evaluate

evaluate = Evaluate(devset= examples, metric=eval_metric, num_threads=6, display_progress=True, display_table=10)


In [226]:
class Ent(dspy.Signature):
    """Extract named entities from the text. Possible entity type: ART, BUILDING, EVENT, LOCATION, ORGANIZATION, OTHER, PERSON, PRODUCT"""
    text = dspy.InputField()
    label = dspy.OutputField(desc='The list of named entities in the text: [{text: the text span, value: the entity label},]', prefix = 'Entities:')

In [221]:
class SimpleEnt(dspy.Module):
    def __init__(self, max_retries=3):
        super().__init__()
        self.prog = dspy.Predict(Ent)
        self.max_retries = max_retries

    def forward(self, text):
        return self.prog(text = text)
        # for attempt in range(self.max_retries):
        #     try:
        #         pred = self.prog(text=text)
                
        #         # Check if we got a valid response
        #         matches = re.findall(r"\[\{.*\}\]", pred.label)
        #         if matches:
        #             return pred
                    
        #     except Exception as e:
        #         print(f"Attempt {attempt + 1} failed: {str(e)}")
                
        #     if attempt < self.max_retries - 1:
        #         time.sleep(1)  # Wait before retrying
                
        # # If all retries failed, return empty prediction
        # return dspy.Prediction(label="[]")

In [227]:
simple_ent = SimpleEnt()

In [224]:
pred = simple_ent(text=example.text)
print("\nTEXT:\n")
print(example.text)

print("\nANSWER:\n")
print(example.label)
print("\nPREDICTION:\n")
print(pred)



TEXT:

In the early 1930s the band moved to the Grill Room of the Taft Hotel in New York ; the band was renamed ``George Hall and His Hotel Taft Orchestra``.

ANSWER:

[{'Grill Room': 'BUILDING'}, {'Taft Hotel': 'BUILDING'}, {'New York': 'LOCATION'}, {'George Hall and His Hotel Taft Orchestra': 'ORGANIZATION'}]

PREDICTION:

Prediction(
    label='[{"text": "Grill Room", "value": "BUILDING"}, {"text": "Taft Hotel", "value": "BUILDING"}, {"text": "New York", "value": "LOCATION"}, {"text": "George Hall", "value": "PERSON"}, {"text": "Hotel Taft Orchestra", "value": "ORGANIZATION"}]'
)


In [193]:
eval_metric(example, pred)

0.6666666666666665

In [25]:
evaluate = Evaluate(devset= examples, metric=eval_metric, num_threads=1, display_progress=True, display_table=10, return_outputs= True, return_all_scores=True, provide_traceback=True)
results = evaluate(simple_ent)
items = []
for sample in results[1]:

    item = {}
    sentence = sample[0]['text']
    label = sample[0]['label']
    if sample[1] == {}:
        pred = {}
    else:
        pred = sample[1]['label']
    item['text'] = sentence
    item['label'] = label
    item['pred'] = pred
    items.append(item)
df_result = pd.DataFrame(data = items)
df_result.to_csv('results/ner/claude-0shot-ner.csv')

Average Metric: 140.15960444783977 / 251  (55.8): 100%|██████████| 251/251 [01:38<00:00,  2.54it/s]


,text,example_label,pred_label,eval_metric
0,In the early 1930s the band moved to the Grill Room of the Taft Hotel in New York ; the band was renamed ``George Hall...,"[{'Grill Room': 'BUILDING'}, {'Taft Hotel': 'BUILDING'}, {'New York': 'LOCATION'}, {'George Hall and His Hotel Taft Orchestra': 'ORGANIZATION'}]","[{""text"": ""Grill Room"", ""value"": ""BUILDING""}, {""text"": ""Taft Hotel"", ""value"": ""BUILDING""}, {""text"": ""New York"", ""value"": ""LOCATION""}, {""text"": ""George Hall"", ""value"": ""PERSON""}, {""text"": ""Hotel Taft Orchestra"", ""value"":...",✔️ [0.667]
1,"The final season of minor league play Elkin Memorial Park saw season attendance of 16,322, an average of 299 per contest.",[{'Elkin Memorial Park': 'LOCATION'}],"[{""text"": ""Elkin Memorial Park"", ""value"": ""BUILDING""}]",
2,"They finished the season 14\u201319, 9\u20139 in C-USA play to finish in seventh place.",[{'C-USA play': 'EVENT'}],"[{""text"": ""C-USA"", ""value"": ""ORGANIZATION""}]",
3,"The B-52 pilot, Major Larry G.Messinger, later recalled,","[{'B-52': 'PRODUCT'}, {'Larry G.Messinger': 'PERSON'}]","[{""text"": ""B-52"", ""value"": ""PRODUCT""}, {""text"": ""Larry G.Messinger"", ""value"": ""PERSON""}]",✔️ [1.000]
4,The Austro-Hungarian Navy built and operated two classes of protected cruisers.,[{'Austro-Hungarian Navy': 'ORGANIZATION'}],"[{""text"": ""Austro-Hungarian Navy"", ""value"": ""ORGANIZATION""}]",✔️ [1.000]
5,Elin Hilderbrand is an American writer mostly of romance novels.,"[{'Elin Hilderbrand': 'PERSON'}, {'American': 'LOCATION'}]","[{""text"": ""Elin Hilderbrand"", ""value"": ""PERSON""}]",✔️ [0.667]
6,"A prototype was fitted in the mid-'60s in a one-off DB5 extended 4'' after the doors and driven by Marek personally, and a normally 6-cylinder...","[{""DB5 extended 4''"": 'PRODUCT'}, {'Marek': 'PERSON'}, {'Aston Martin DB7': 'PRODUCT'}, {'V8 unit': 'PRODUCT'}]","[{""text"": ""DB5"", ""value"": ""PRODUCT""}, {""text"": ""Marek"", ""value"": ""PERSON""}, {""text"": ""Aston Martin DB7"", ""value"": ""PRODUCT""}]",✔️ [0.571]
7,"He has caught the attention of major publications and media outlets such as CNN, The Huffington Post, Hypebeast, The Guardian, Juxtapoz Magazine, Daily Mail, NRDC,...","[{'CNN': 'ORGANIZATION'}, {'The Huffington Post': 'ORGANIZATION'}, {'Hypebeast': 'ORGANIZATION'}, {'The Guardian': 'ORGANIZATION'}, {'Juxtapoz Magazine': 'ORGANIZATION'}, {'Daily Mail, NRDC': 'ORGANIZATION'}, {'Aljazeera': 'ORGANIZATION'}, {'Hi-Fructose Magazine': 'ORGANIZATION'}]","[{""text"": ""CNN"", ""value"": ""ORGANIZATION""}, {""text"": ""The Huffington Post"", ""value"": ""ORGANIZATION""}, {""text"": ""Hypebeast"", ""value"": ""ORGANIZATION""}, {""text"": ""The Guardian"", ""value"": ""ORGANIZATION""}, {""text"": ""Juxtapoz Magazine"", ""value"": ""ORGANIZATION""}, {""text"":...",✔️ [0.824]
8,The Cnidaria are a group of animals found exclusively in aquatic and mostly marine environments.,[{'Cnidaria': 'OTHER'}],"[{""text"": ""Cnidaria"", ""value"": ""OTHER""}]",✔️ [1.000]
9,"The Ninth suffered a serious defeat at the Battle of Camulodunum under Quintus Petillius Cerialis in the rebellion of Boudica (61), when most of the...","[{'Camulodunum': 'EVENT'}, {'Quintus Petillius Cerialis': 'PERSON'}, {'Boudica': 'EVENT'}, {'Camulodunum': 'LOCATION'}, {'Colchester': 'LOCATION'}]",[],


In [20]:
def evaluate_modified_set(ds, program):
    examples = [
    dspy.Example({ 
                  "text" : remove_space(r["modified_text"]), 
                  "label": str(r['modified_label']),
                  "original_text": remove_space(r['original_text']),
                  "original_label": str(r['original_label']),
                  "index": r['index'],
                  "type": r['subtype'] if 'subtype' in r else None
                #   "original_label": str(r['original_label'])
                }
                  ).with_inputs("text")
    
    for r in ds
    ]
    evaluate = Evaluate(devset= examples, metric=eval_metric, num_threads=1, display_progress=True, display_table=1, return_outputs= True, return_all_scores=True, provide_traceback=True)
    results = evaluate(program)
    return results

In [21]:
class Ent(dspy.Signature):
    """Extract named entities from the text. Possible entity type: ART, BUILDING, EVENT, LOCATION, ORGANIZATION, OTHER, PERSON, PRODUCT"""
    text = dspy.InputField()
    label = dspy.OutputField(desc='The list of named entities in the text: [{"text": the text span, "value": the entity label},]', prefix = 'Entities:')

class SimpleEnt(dspy.Module):
    def __init__(self, max_retries=3):
        super().__init__()
        self.prog = dspy.Predict(Ent)
        self.max_retries = max_retries

    def forward(self, text):
        for attempt in range(self.max_retries):
            try:
                pred = self.prog(text=text)
                
                # Check if we got a valid response
                matches = re.findall(r"\[\{.*\}\]", pred.label)
                if matches:
                    return pred
                    
            except Exception as e:
                print(f"Attempt {attempt + 1} failed: {str(e)}")
                
            if attempt < self.max_retries - 1:
                time.sleep(1)  # Wait before retrying
                
        # If all retries failed, return empty prediction
        return dspy.Prediction(label="[]")
simple_ent = SimpleEnt()

In [22]:
import glob
import difflib


In [26]:
# Configure GPT-4 as the language model
original_pred_ds = pd.read_csv('results/ner/claude-0shot-ner.csv', index_col=False)
original_pred_ds['text'] = original_pred_ds['text'].apply(lambda x: remove_space(x.encode('utf-8').decode('unicode-escape')))  # Replace 'your_function' with the actual function

# Get specific json files we want to process
json_files = glob.glob('../data/modified_data/ner/*_100.json')

for json_file in json_files:
    # if 'negation' not in json_file:
    #     continue
    # Load the json file
    print(json_file)
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    results = evaluate_modified_set(data,simple_ent)

    # Convert results to dataframe
    items = []
    for sample in results[1]:
        item = {}
        sentence = sample[0]['text']
        label = sample[0]['label'] 
        # print(sample)
        if sample[1] == {}:
            pred = "[]"
        else:
            pred = sample[1]['label']
        item['text'] = sentence
        # print(label)
        # item['modified_label'] = [{entity['text']: entity['value']} for entity in label]
        item['modified_label'] = label
        pred = extract_prediction(pred)
        item['modified_pred'] = pred
        item['modified_pred'] = [{entity['text']: entity['value']} for entity in pred]
        original_text = sample[0]['original_text'].encode('utf-8').decode('unicode-escape')
        item['original_text'] = original_text
        index = sample[0]['index']
        # Find the best match for the original_text in the original_pred_ds['text'] using difflib
        matches = original_pred_ds['pred'].iloc[index] if index < len(original_pred_ds) else []
        item['original_pred'] = matches if matches else '[]'

        item['original_label'] = sample[0]['original_label']
        
        # Check if original_label is NaN and assign modified_label if it is
        if pd.isna(item['original_label']):
            item['original_label'] = item['modified_label']
        item['type'] = sample[0]['type']
        items.append(item)
    
    df_result = pd.DataFrame(data=items)
    
    # Save results with filename based on input json
    output_filename = f"results/ner/claude-0shot-{json_file.split('/')[-1].replace('.json', '')}_new.csv"
    df_result.to_csv(output_filename)


../preprocessing/data_after_phase2/rahmad/casual_100.json
Average Metric: 40.966849816849816 / 92  (44.5): 100%|██████████| 92/92 [00:48<00:00,  1.89it/s]


,text,example_label,original_text,original_label,index,type,pred_label,eval_metric
0,He produced Kim Fowley and BMX Bandits' ``Hidden Agenda At the Thirteenth Not`` for Receiver Records.,"[{'text': 'Kim Fowley', 'value': 'PERSON'}, {'text': 'BMX Bandits', 'value': 'ORGANIZATION'}, {'text': 'Receiver Records', 'value': 'ORGANIZATION'}, {'text': 'Hidden Agenda At the Thirteenth Not', 'value': 'ART'}]",He went on to produce Kim Fowley and the BMX Bandits (band) Receiver Records' album ``Hidden Agenda At the Thirteenth Not``.,"[{'Kim Fowley': 'PERSON'}, {'BMX Bandits': 'ORGANIZATION'}, {'Receiver Records': 'ORGANIZATION'}, {'Hidden Agenda At the Thirteenth Not': 'ART'}]",25,None,"[{""text"": ""Kim Fowley"", ""value"": ""PERSON""}, {""text"": ""BMX Bandits"", ""value"": ""ORGANIZATION""}, {""text"": ""Hidden Agenda At the Thirteenth Not"", ""value"": ""ART""}, {""text"": ""Receiver Records"", ""value"": ""ORGANIZATION""}]",✔️ [1.000]


../preprocessing/data_after_phase2/rahmad/discourse_100.json
Average Metric: 39.84321789321789 / 72  (55.3): 100%|██████████| 72/72 [00:34<00:00,  2.10it/s] 


,text,example_label,original_text,original_label,index,type,pred_label,eval_metric
0,"Santa is actually innocent of the crime, which was instead masterminded by scheming relative Cousin Mel, who is mentioned briefly in the song, made into...","[{'text': 'Santa', 'value': 'PERSON'}, {'text': 'Cousin Mel', 'value': 'PERSON'}]","Moreover, Santa is actually innocent of the crime, which was instead masterminded by scheming relative Cousin Mel, who is mentioned briefly in the song but...","[{'Santa': 'PERSON'}, {'Cousin Mel': 'PERSON'}]",43,delete,"[{""text"": ""Santa"", ""value"": ""PERSON""}, {""text"": ""Cousin Mel"", ""value"": ""PERSON""}]",✔️ [1.000]


../preprocessing/data_after_phase2/rahmad/compound_word_100.json
Average Metric: 46.74523809523808 / 86  (54.4): 100%|██████████| 86/86 [00:40<00:00,  2.13it/s] 


...

,text,example_label,original_text,original_label,index,type,pred_label,eval_metric
0,The government announced a national funeral and a day of national mourning.,[],The government announced a state funeral and a day of national mourning.,[],46,None,[],


../preprocessing/data_after_phase2/rahmad/active_to_passive_100.json
Average Metric: 46.078571428571415 / 81  (56.9): 100%|██████████| 81/81 [00:36<00:00,  2.22it/s]


,text,example_label,original_text,original_label,index,type,pred_label,eval_metric
0,"Genre classics are focused on by Back to Basics, with older movies and underground cult films being shown, and with occasional stabs at mainstream and...","[{'text': 'Back to Basics', 'value': 'ORGANIZATION'}]","Back to Basics focuses on genre classics, showing older movies and underground cult films, with occasional stabs at mainstream and newer genre pieces.",[{'Back to Basics': 'ORGANIZATION'}],56,None,"[{""text"": ""Back to Basics"", ""value"": ""ORGANIZATION""}]",✔️ [1.000]


../preprocessing/data_after_phase2/rahmad/negation_100.json
Average Metric: 53.59733893557422 / 110  (48.7): 100%|██████████| 110/110 [00:56<00:00,  1.95it/s]


,text,example_label,original_text,original_label,index,type,pred_label,eval_metric
0,"It is developed by no company, neither Golaem nor any other (created in Rennes in 2009).","[{'text': 'Golaem', 'value': 'ORGANIZATION'}, {'text': 'Rennes', 'value': 'LOCATION'}]","It is developed by Golaem, a France -based software company (created in Rennes in 2009).","[{'Golaem': 'ORGANIZATION'}, {'France': 'LOCATION'}, {'Rennes': 'LOCATION'}]",28,absolute,"[{""text"": ""Golaem"", ""value"": ""ORGANIZATION""}, {""text"": ""Rennes"", ""value"": ""LOCATION""}]",✔️ [1.000]


In [258]:
from scipy import stats

In [262]:
result_files = glob.glob('results/ner/claude-0shot-*_100.csv')

aggregated_results = []

def convert_string_to_entities(entity_str):
    """Convert string representation of entities to proper format"""
    if isinstance(entity_str, str):
        try:
            # Convert string to list of dicts
            entities = ast.literal_eval(entity_str)
            # Handle nested lists by flattening
            if isinstance(entities, list):
                # Handle double nested lists
                if len(entities) > 0 and isinstance(entities[0], list):
                    entities = entities[0]
                # Handle list of dicts with text/value format
                if len(entities) > 0 and isinstance(entities[0], dict):
                    # Handle format with text/value keys
                    if 'text' in entities[0]:
                        return entities
                    # Handle format with single key-value pair
                    if len(entities[0]) == 1:
                        converted = []
                        for e in entities:
                            for text, value in e.items():
                                converted.append({'text': text, 'value': value})
                        return converted
                    # Handle format with multiple key-value pairs
                    converted = []
                    for e in entities:
                        for text, value in e.items():
                            if isinstance(value, str):
                                converted.append({'text': text, 'value': value})
                    return converted
            return entities
        except:
            return []
    return entity_str

for file in result_files:
    # Extract modification type from filename
    mod_type = file.split('-')[-1].replace('.csv','')
    print(mod_type)
    # Read results file
    df = pd.read_csv(file)

    # Collect all predictions and labels for micro F1 calculation
    all_original_labels = []
    all_original_preds = []
    all_modified_labels = []
    all_modified_preds = []

    for idx, row in df.iterrows():
        # Convert string representations to proper format
        original_label = convert_string_to_entities(row['original_label'])
        original_pred = convert_string_to_entities(row['original_pred'])
        modified_label = convert_string_to_entities(row['modified_label'])
        modified_pred = convert_string_to_entities(row['modified_pred'])

        # Append to combined lists
        all_original_labels.extend(original_label)
        all_original_preds.extend(original_pred)
        all_modified_labels.extend(modified_label)
        all_modified_preds.extend(modified_pred)

    # Calculate micro F1 scores using calculate_f1_ent
    original_precision, original_recall, original_f1 = calculate_f1_ent(all_original_labels, all_original_preds)
    modified_precision, modified_recall, modified_f1 = calculate_f1_ent(all_modified_labels, all_modified_preds)
    
    # Calculate the difference between original and modified F1 scores
    difference = -round(original_f1 - modified_f1, 2)
    
    # Calculate percentage difference with respect to original F1
    pct_difference = -round((original_f1 - modified_f1) / original_f1 * 100, 2) if original_f1 != 0 else 0
    
    # Perform t-test between original and modified predictions
    t_stat, p_value = stats.ttest_ind(
        (df['original_pred'] == df['original_label']).astype(float),
        (df['modified_pred'] == df['modified_label']).astype(float)
    )
    
    aggregated_results.append({
        'task': 'named_entity_recognition',
        'modification': mod_type,
        'original_res': round(original_f1 , 2),  # Convert to percentage
        'modified_res': round(modified_f1 , 2),  # Convert to percentage
        'difference': difference,
        'pct_difference': pct_difference,
        'p_value': p_value,
        'original_precision': round(original_precision , 2),  # Convert to percentage
        'original_recall': round(original_recall , 2),  # Convert to percentage
        'modified_precision': round(modified_precision , 2),  # Convert to percentage
        'modified_recall': round(modified_recall , 2)  # Convert to percentage
    })

# Create final results dataframe
results_df = pd.DataFrame(aggregated_results)

# Sort the results based on modification_name
modification_name = ['temporal_bias_100', 'geographical_bias_100','length_bias_100', 'typo_bias_100', 'capitalization_100', 'punctuation_100', 'derivation_100', 'compound_word_100','active_to_passive_100','grammatical_role_100', 'coordinating_conjunction_100', 'concept_replacement_100','negation_100','discourse_100','sentiment_100','casual_100', 'dialectal_100']
results_df['modification'] = pd.Categorical(results_df['modification'], categories=modification_name, ordered=True)
results_df = results_df.sort_values(by='modification')

# Calculate averages across all modifications
avg_original = results_df['original_res'].mean()
avg_modified = results_df['modified_res'].mean()
avg_difference = avg_original - avg_modified
avg_pct_difference = results_df['pct_difference'].mean()
avg_orig_precision = results_df['original_precision'].mean()
avg_orig_recall = results_df['original_recall'].mean()
avg_mod_precision = results_df['modified_precision'].mean()
avg_mod_recall = results_df['modified_recall'].mean()

# Add averages as a new row
results_df.loc[len(results_df)] = {
    'task': 'named_entity_recognition',
    'modification': 'average',
    'original_res': round(avg_original, 2),
    'modified_res': round(avg_modified, 2),
    'difference': -round(avg_difference, 2),
    'pct_difference': round(avg_pct_difference, 2),
    'p_value': None,
    'original_precision': round(avg_orig_precision, 2),
    'original_recall': round(avg_orig_recall, 2),
    'modified_precision': round(avg_mod_precision, 2),
    'modified_recall': round(avg_mod_recall, 2)
}

print("\n")
results_df.to_csv('results/ner/claude-DP.csv')

# Apply styling to highlight rows where original_res > modified_res and significant p-values
def highlight_drops_and_significance(row):
    colors = [''] * len(row)
    if row['original_res'] > row['modified_res']:
        colors = ['background-color: red'] * len(row)
        # If p-value < 0.05, add bold text
        if 'p_value' in row and row['p_value'] is not None and row['p_value'] < 0.05:
            colors = ['background-color: red; font-weight: bold'] * len(row)
    return colors

results_df.round(2).style.apply(highlight_drops_and_significance, axis=1)


casual_100
punctuation_100
sentiment_100
geographical_bias_100
length_bias_100
grammatical_role_100
compound_word_100
capitalization_100
active_to_passive_100
temporal_bias_100
typo_bias_100
coordinating_conjunction_100
dialectal_100
concept_replacement_100
derivation_100
negation_100
discourse_100




/var/folders/10/sw0slq6565ddbg_lqxxhpqzh0000gn/T/ipykernel_43305/3061431129.py:115: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df.loc[len(results_df)] = {


,task,modification,original_res,modified_res,difference,pct_difference,p_value,original_precision,original_recall,modified_precision,modified_recall
9,named_entity_recognition,temporal_bias_100,0.800000,0.770000,-0.030000,-4.020000,1.000000,0.810000,0.800000,0.810000,0.740000
3,named_entity_recognition,geographical_bias_100,0.780000,0.830000,0.050000,6.530000,nan,0.760000,0.800000,0.830000,0.830000
4,named_entity_recognition,length_bias_100,0.840000,0.760000,-0.070000,-8.560000,0.790000,0.810000,0.860000,0.800000,0.730000
10,named_entity_recognition,typo_bias_100,0.780000,0.780000,0.000000,0.100000,1.000000,0.780000,0.770000,0.790000,0.770000
7,named_entity_recognition,capitalization_100,0.810000,0.760000,-0.050000,-5.730000,0.760000,0.810000,0.800000,0.780000,0.740000
1,named_entity_recognition,punctuation_100,0.800000,0.660000,-0.140000,-17.250000,1.000000,0.800000,0.800000,0.750000,0.590000
14,named_entity_recognition,derivation_100,0.810000,0.800000,-0.010000,-1.360000,1.000000,0.800000,0.830000,0.790000,0.810000
6,named_entity_recognition,compound_word_100,0.810000,0.730000,-0.080000,-10.270000,0.810000,0.820000,0.800000,0.810000,0.660000
8,named_entity_recognition,active_to_passive_100,0.820000,0.720000,-0.100000,-12.400000,1.000000,0.800000,0.850000,0.790000,0.660000
5,named_entity_recognition,grammatical_role_100,0.840000,0.810000,-0.030000,-3.920000,1.000000,0.840000,0.840000,0.830000,0.790000
